In [17]:
import os
import glob

import numpy as np
import pandas as pd
import zipfile
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
from collections import defaultdict

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score

# Load Data

In [24]:
training_traj = np.load(f'/root/Calibration/PhyPhox/Data/training_traj.npy')[:, :, :-1]
# validation_traj = np.load(f'/root/Calibration/PhyPhox/Data/validation_traj.npy')[:, :, :-1]
# testing_traj = np.load(f'/root/Calibration/PhyPhox/Data/testing_traj.npy')[:, :, :-1]
validation_traj = np.load(f'./User2_data.npy', allow_pickle=True)

In [40]:
posture_data = defaultdict(list)

valid_data = validation_traj.item()
for path_name in valid_data.keys():
    for posture_name in valid_data[path_name].keys():
        data = valid_data[path_name][posture_name]
        pos = data[:, [0, 1]] + np.array((1, 1))
        mag = data[:, [12, 13, 14]]
        
        pos_mag = np.concatenate((pos, mag), axis=-1)
        posture_data[posture_name].append(pos_mag)

In [26]:
training_traj.shape, validation_traj.shape

((100, 100, 5), ())

In [28]:
training_traj[0][0]

array([  4.92      ,   1.2       ,  14.78095265, -18.16837343,
       -21.63703396])

In [7]:
# def expand(trajs):
#     new_trajs = []
#     for traj in trajs:
#         pos = traj[:, 0:2]
#         T = np.sqrt(np.sum(np.power(traj[:, 2:5], 2), axis=-1, keepdims=1))
#         H = np.sqrt(np.sum(np.power(traj[:, 2:4], 2), axis=-1, keepdims=1))
#         V = traj[:, 4:5]
        
#         new_traj = np.concatenate((pos, H, V, T), axis=-1)
        
#         new_trajs.append(new_traj)
        
#     return np.array(new_trajs)

# PreProcess

In [31]:
def split_traj(trajs, length=10):
    sp_traj = []
    for traj in trajs:
        for i in range(len(traj) - length + 1):
            sp_traj.append(traj[i:i+length])
    return np.array(sp_traj)

In [41]:
prep_training_traj = split_traj(training_traj)
prep_valid_front_pocket_traj = split_traj(posture_data['front_pocket'])
prep_valid_horizontal_traj = split_traj(posture_data['horizontal'])
prep_valid_pocket_traj = split_traj(posture_data['pocket'])
prep_valid_swing_traj = split_traj(posture_data['swing'])
prep_valid_target_traj = split_traj(posture_data['target'])

In [42]:
prep_training_traj.shape, prep_valid_front_pocket_traj.shape, prep_valid_horizontal_traj.shape, prep_valid_pocket_traj.shape, prep_valid_swing_traj.shape, prep_valid_target_traj.shape

((9100, 10, 5),
 (156, 10, 5),
 (156, 10, 5),
 (156, 10, 5),
 (156, 10, 5),
 (156, 10, 5))

# Dataloader

In [20]:
class PairDataset(Dataset):
    def __init__(self, pos, rcplot):
        self.pos = torch.tensor(pos, dtype=torch.float)
        self.rcplot = torch.tensor(rcplot, dtype=torch.float)

    def __len__(self):
        return len(self.pos)

    def __getitem__(self, idx):
        return self.pos[idx], self.rcplot[idx]

In [21]:
train_loader = DataLoader(PairDataset(training_pos, training_rcplot), batch_size=64, shuffle=True)
valid_loader = DataLoader(PairDataset(validation_pos, validation_rcplot), batch_size=64, shuffle=False)
test_loader = DataLoader(PairDataset(testing_pos, testing_rcplot), batch_size=64, shuffle=False)

# Model Architecture

In [22]:
class RPCNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.cnn1 = nn.Sequential(
            nn.Conv2d(3, 128, 2, padding=3),
            nn.SELU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.25)
        )
        
        self.cnn2 = nn.Sequential(
            nn.Conv2d(128, 128, 2, padding=3),
            nn.SELU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.25)
        )
        
        self.cnn3 = nn.Sequential(
            nn.Conv2d(128, 256, 2, padding=2),
            nn.SELU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.25)
        )
        
        self.linear = nn.Sequential(
            nn.Linear(10, 4096),
        )
        
        self.gru = nn.GRU(4096, 4096, batch_first=True)
        
        self.last = nn.Sequential(
            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(1024, 768),
            nn.ReLU(),
            nn.Dropout(0.5),
            
            nn.Linear(768, 512),
            nn.ReLU(),
            
            nn.Linear(512, 2),
        )
        
    def forward(self, rcplot):
        bs, channel, seq_len, seq_len = rcplot.shape

        # up part
        z = self.cnn1(rcplot)
        z = self.cnn2(z)
        z = self.cnn3(z)
        z = z.reshape((bs, -1))
        
        # bottom part
        V = self.linear(rcplot.reshape((bs, channel * seq_len, -1)))
        att = V @ z[:, :, None]
        
        r = torch.sum(F.softmax(att, dim=1) * V, dim=1)
        r, _ = self.gru(z[:, None, :], r[None, :, :])
        
        # last part
        z = z + r[:, 0]
        z = self.last(z)
        
        return z
        

In [23]:
def train(dataloader):
    model.train()

    mse_loss = []

    for gt_pos, replots in dataloader:
        optimizer.zero_grad()
        
        gt_pos = gt_pos.to(device)
        replots = replots.to(device)

        # localization
        pred_pos = model(replots)
        
        # loss
        loss = MSE_Loss(pred_pos, gt_pos)
        
        # backward
        loss.backward()
        optimizer.step()
        
        # record loss
        mse_loss.append(loss.item())
    
    return np.mean(mse_loss)


In [24]:
def evaluate(dataloader):
    model.eval()

    mse_loss = []

    with torch.no_grad():
        for gt_pos, replots in dataloader:
            optimizer.zero_grad()
            
            gt_pos = gt_pos.to(device)
            replots = replots.to(device)

            # localization
            pred_pos = model(replots)

            # loss
            loss = MSE_Loss(pred_pos, gt_pos)

            # record loss
            mse_loss.append(loss.item())
    
    return np.mean(mse_loss)


In [25]:
EPOCH = 6000
device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")
MSE_Loss = torch.nn.MSELoss()

In [26]:
torch.nn.MSELoss()(torch.tensor([[3, 5]], dtype=torch.float), torch.tensor([[5, 4]], dtype=torch.float))

tensor(2.5000)

In [27]:
model = RPCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005)

In [28]:
output = model(torch.empty(16, 3, 10, 10).to(device))
output.shape

torch.Size([16, 2])

In [29]:
from torchinfo import summary
summary(model, input_size=[(1, 3, 10, 10)])

Layer (type:depth-idx)                   Output Shape              Param #
RPCNN                                    [1, 2]                    --
├─Sequential: 1-1                        [1, 128, 7, 7]            --
│    └─Conv2d: 2-1                       [1, 128, 15, 15]          1,664
│    └─SELU: 2-2                         [1, 128, 15, 15]          --
│    └─MaxPool2d: 2-3                    [1, 128, 7, 7]            --
│    └─Dropout2d: 2-4                    [1, 128, 7, 7]            --
├─Sequential: 1-2                        [1, 128, 6, 6]            --
│    └─Conv2d: 2-5                       [1, 128, 12, 12]          65,664
│    └─SELU: 2-6                         [1, 128, 12, 12]          --
│    └─MaxPool2d: 2-7                    [1, 128, 6, 6]            --
│    └─Dropout2d: 2-8                    [1, 128, 6, 6]            --
├─Sequential: 1-3                        [1, 256, 4, 4]            --
│    └─Conv2d: 2-9                       [1, 256, 9, 9]            131,328
│  

In [ ]:
for epoch in range(EPOCH):
    
    train_loss = train(train_loader)
    valid_loss = evaluate(valid_loader)
    test_loss = evaluate(test_loader)
    
    if (epoch + 1) % 1 == 0:
        ep = str(epoch + 1).zfill(5)
        print(f'{ep:>5}: train loss: {train_loss: 2.3f}   valid loss: {valid_loss: 2.3f}   test loss: {test_loss: 2.3f}')

In [ ]:
03909: train loss:  0.003   valid loss:  0.066   test loss:  0.946
03910: train loss:  0.002   valid loss:  0.066   test loss:  0.939

In [ ]:
03909: train loss:  0.005   valid loss:  0.217   test loss:  1.436
03910: train loss:  0.004   valid loss:  0.205   test loss:  1.445